In [1]:
#!/usr/bin/env python3
"""
BLOCK 0: Exploratory Data Analysis (EDA) - (Revised)

This script performs 10 essential EDA checks on the raw
FD001 dataset (train, test, RUL files).

It generates and saves plots to the 'eda_plots_fd001' directory
to visualize:
1.  Engine Lifespan Distribution
2.  Individual Sensor Traces (Raw)
3.  Sensor vs. Sensor Correlation Heatmap
4.  Train vs. Test Sensor Data Distributions
5.  RUL Target Distributions
6.  Operating Setting Validation (NEW)
7.  Sensor vs. RUL Correlation Heatmap (NEW - CRITICAL)
8.  Scatter Plots for Top Sensors vs. RUL (NEW - CRITICAL)
9.  Normalized Degradation Traces (NEW)
10. Start-of-Life vs. End-of-Life Sensor Distributions (NEW)
"""

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ---------------------------
# CONFIGURATION
# ---------------------------

# --- Input Files ---
TRAIN_FILE = "train_FD003.txt"
TEST_FILE = "test_FD003.txt"
RUL_FILE = "RUL_FD003.txt"

# --- Output Directory ---
PLOT_DIR = "eda_plots_fd003"

# --- Data Columns (from CMAPSS description) ---
SENSOR_COLS = [f's_{i}' for i in range(1, 22)]
OP_SETTING_COLS = [f'os_{i}' for i in range(1, 4)]
COL_NAMES = ['engine_id', 'cycle'] + OP_SETTING_COLS + SENSOR_COLS

# --- Feature Selection (from your Block 1) ---
DROP_SENSORS = ['s_1', 's_5', 's_10', 's_16', 's_18', 's_19']
USEFUL_SENSORS = [s for s in SENSOR_COLS if s not in DROP_SENSORS]

# ---------------------------
# UTILITY FUNCTIONS
# ---------------------------

def safe_read_cmapss(path):
    """Reads the space-delimited CMAPSS files."""
    try:
        return pd.read_csv(path, sep=r'\s+', header=None, engine='python')
    except FileNotFoundError:
        print(f"FATAL ERROR: File not found: {path}")
        print("Please ensure train_FD003.txt, test_FD003.txt, and RUL_FD003.txt are present.")
        return None
    except Exception as e:
        print(f"Error reading {path}: {e}")
        return None

def load_data():
    """Loads all three raw data files."""
    print("Loading data...")
    train_df = safe_read_cmapss(TRAIN_FILE)
    test_df = safe_read_cmapss(TEST_FILE)
    rul_df = safe_read_cmapss(RUL_FILE)

    if train_df is None or test_df is None or rul_df is None:
        return None, None, None, None

    # Assign column names
    train_df.columns = COL_NAMES[:train_df.shape[1]]
    test_df.columns = COL_NAMES[:test_df.shape[1]]

    # --- Pre-calculate RUL for the training set ---
    # This is essential for many plots
    max_cycles = train_df.groupby('engine_id')['cycle'].max().reset_index()
    max_cycles.columns = ['engine_id', 'max_cycle']
    train_df = train_df.merge(max_cycles, on='engine_id', how='left')
    train_df['RUL'] = train_df['max_cycle'] - train_df['cycle']
    train_df['normalized_cycle'] = train_df['cycle'] / train_df['max_cycle']
    train_df.drop('max_cycle', axis=1, inplace=True)

    print(f"Training data shape: {train_df.shape}")
    print(f"Test data shape: {test_df.shape}")

    return train_df, test_df, rul_df, max_cycles['max_cycle']

# ---------------------------
# PLOTTING FUNCTIONS
# ---------------------------

def plot_1_lifespan_histogram(max_cycles_data, save_path):
    """Plots a histogram of the max cycle (total life) for all engines."""
    print("1. Plotting engine lifespan distribution...")
    try:
        plt.figure(figsize=(10, 6))
        sns.histplot(max_cycles_data, kde=True, bins=30)
        plt.title('Engine Lifespan Distribution (Train Set)')
        plt.xlabel('Total Cycles (Max Life)')
        plt.ylabel('Number of Engines')
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()
    except Exception as e:
        print(f"  Failed plot 1: {e}")

def plot_2_sensor_traces(train_df, save_path, num_engines=5):
    """Plots all 21 sensor traces for a few sample engines."""
    print(f"2. Plotting sensor traces for {num_engines} sample engines...")
    try:
        all_engine_ids = train_df['engine_id'].unique()
        sample_ids = np.random.choice(all_engine_ids, min(num_engines, len(all_engine_ids)), replace=False)
        sample_df = train_df[train_df['engine_id'].isin(sample_ids)]

        fig, axes = plt.subplots(7, 3, figsize=(20, 25))
        axes = axes.flatten()

        for i, sensor in enumerate(SENSOR_COLS):
            ax = axes[i]
            sns.lineplot(data=sample_df, x='cycle', y=sensor, hue='engine_id', ax=ax, legend=False)
            ax.set_title(f'Sensor: {sensor}')
            ax.set_ylabel('Sensor Value')
            ax.set_xlabel('Cycle')

            if sensor in DROP_SENSORS:
                ax.set_facecolor('#ffcccc')
                ax.set_title(f'Sensor: {sensor} (DROPPED)', color='red')

        plt.suptitle('Full-Life Sensor Traces (Sample Engines)', fontsize=24, y=1.02)
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()
    except Exception as e:
        print(f"  Failed plot 2: {e}")

def plot_3_correlation_heatmap(train_df, save_path):
    """Plots a heatmap of sensor correlations."""
    print("3. Plotting sensor correlation heatmap...")
    try:
        corr_matrix = train_df[SENSOR_COLS].corr()
        plt.figure(figsize=(16, 12))
        sns.heatmap(corr_matrix, annot=True, fmt='.1f', cmap='coolwarm', annot_kws={"size": 8})
        plt.title('Sensor vs. Sensor Correlation Heatmap (Train Set)')
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()
    except Exception as e:
        print(f"  Failed plot 3: {e}")

def plot_4_train_test_distributions(train_df, test_df, save_path):
    """Plots overlapping KDEs for train vs. test sensor values."""
    print("4. Plotting Train vs. Test sensor distributions...")
    try:
        num_sensors = len(USEFUL_SENSORS)
        num_cols = 3
        num_rows = int(np.ceil(num_sensors / num_cols))

        fig, axes = plt.subplots(num_rows, num_cols, figsize=(num_cols * 6, num_rows * 4))
        axes = axes.flatten()

        for i, sensor in enumerate(USEFUL_SENSORS):
            ax = axes[i]
            sns.kdeplot(train_df[sensor], ax=ax, label='Train', color='blue', fill=True)
            sns.kdeplot(test_df[sensor], ax=ax, label='Test', color='red', fill=True)
            ax.set_title(f'Distribution for: {sensor}')
            ax.legend()

        for j in range(i + 1, len(axes)):
            axes[j].set_visible(False)

        plt.suptitle('Train vs. Test Data Distributions (Useful Sensors)', fontsize=24, y=1.02)
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()
    except Exception as e:
        print(f"  Failed plot 4: {e}")

def plot_5_rul_target_distributions(train_df, rul_df, save_path):
    """Plots the RUL distributions for the training data (all points)
       and the test data (final points)."""
    print("5. Plotting RUL target distributions...")
    try:
        train_rul = train_df['RUL']
        test_rul = rul_df[0]

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

        sns.histplot(train_rul, ax=ax1, bins=50, kde=True)
        ax1.set_title('Train RUL Distribution (All Data Points)')
        ax1.set_xlabel('RUL (cycles)')
        ax1.set_ylabel('Frequency')

        sns.histplot(test_rul, ax=ax2, bins=30, kde=True)
        ax2.set_title('Test RUL Distribution (Final Points)')
        ax2.set_xlabel('RUL (cycles)')
        ax2.set_ylabel('Frequency')

        plt.suptitle('RUL Target Distributions', fontsize=20)
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()
    except Exception as e:
        print(f"  Failed plot 5: {e}")

def plot_6_op_setting_validation(train_df, test_df, save_path):
    """Validates that operating settings are constant."""
    print("6. Plotting Operating Setting validation...")
    try:
        fig, axes = plt.subplots(2, 3, figsize=(18, 8))

        for i, col in enumerate(OP_SETTING_COLS):
            sns.histplot(train_df[col], ax=axes[0, i], discrete=True)
            axes[0, i].set_title(f'Train {col} Distribution')

            sns.histplot(test_df[col], ax=axes[1, i], discrete=True)
            axes[1, i].set_title(f'Test {col} Distribution')

        plt.suptitle('Operating Setting Validation (FD001 should be single-mode)', fontsize=20)
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()
    except Exception as e:
        print(f"  Failed plot 6: {e}")

def plot_7_rul_correlation_heatmap(train_df, save_path):
    """Calculates and plots the correlation between useful sensors and RUL."""
    print("7. Plotting Sensor vs. RUL correlation heatmap...")
    try:
        # We only care about the features we actually use + RUL
        rul_corr_df = train_df[USEFUL_SENSORS + ['RUL']]
        corr_matrix = rul_corr_df.corr()

        # Get the RUL correlation series, drop RUL itself, and sort
        rul_corr = corr_matrix['RUL'].drop('RUL').sort_values(ascending=False)

        plt.figure(figsize=(12, 8))
        sns.barplot(x=rul_corr.values, y=rul_corr.index, palette='vlag')
        plt.title('Feature Correlation with RUL (Train Set)')
        plt.xlabel('Pearson Correlation Coefficient')
        plt.ylabel('Sensor')
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()

        # Return the top 4 correlated sensors for the next plot
        top_4_sensors = rul_corr.abs().nlargest(4).index.tolist()
        return top_4_sensors

    except Exception as e:
        print(f"  Failed plot 7: {e}")
        return None

def plot_8_sensor_rul_scatter(train_df, top_sensors, save_path):
    """Plots scatter plots for the top 4 sensors vs. RUL."""
    if top_sensors is None:
        return
    print("8. Plotting scatter plots for top 4 sensors vs. RUL...")
    try:
        # Sample to avoid overplotting
        sample_df = train_df.sample(n=min(5000, len(train_df)), random_state=42)

        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        axes = axes.flatten()

        for i, sensor in enumerate(top_sensors):
            ax = axes[i]
            sns.regplot(data=sample_df, x=sensor, y='RUL', ax=ax,
                        scatter_kws={'alpha': 0.1, 's': 10}, line_kws={'color': 'red'})
            ax.set_title(f'{sensor} vs. RUL')

        plt.suptitle('Top Predictive Sensors vs. RUL (Sampled)', fontsize=20)
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()
    except Exception as e:
        print(f"  Failed plot 8: {e}")

def plot_9_normalized_degradation(train_df, top_sensors, save_path, num_engines=10):
    """Plots normalized degradation traces for top sensors."""
    if top_sensors is None:
        return
    print("9. Plotting normalized degradation traces...")
    try:
        # Sample engines
        all_engine_ids = train_df['engine_id'].unique()
        sample_ids = np.random.choice(all_engine_ids, min(num_engines, len(all_engine_ids)), replace=False)
        sample_df = train_df[train_df['engine_id'].isin(sample_ids)]

        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        axes = axes.flatten()

        for i, sensor in enumerate(top_sensors):
            ax = axes[i]
            sns.lineplot(data=sample_df, x='normalized_cycle', y=sensor, hue='engine_id', ax=ax, legend=False)
            ax.set_title(f'Normalized Degradation: {sensor}')
            ax.set_xlabel('Normalized Cycle (0=Start, 1=Failure)')
            ax.set_ylabel('Sensor Value')

        plt.suptitle(f'Normalized Traces for Top 4 Sensors ({num_engines} Engines)', fontsize=20)
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()
    except Exception as e:
        print(f"  Failed plot 9: {e}")

def plot_10_life_start_vs_end_boxplots(train_df, top_sensors, save_path, cycle_thresh=30):
    """Compares sensor distributions at start vs. end of life."""
    if top_sensors is None:
        return
    print("10. Plotting Start vs. End of Life distributions...")
    try:
        # Create categories for start and end of life
        start_life = train_df[train_df['cycle'] <= cycle_thresh].copy()
        start_life['Life_Stage'] = 'Start of Life'

        end_life = train_df[train_df['RUL'] <= cycle_thresh].copy()
        end_life['Life_Stage'] = 'End of Life'

        comparison_df = pd.concat([start_life, end_life])

        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        axes = axes.flatten()

        for i, sensor in enumerate(top_sensors):
            ax = axes[i]
            sns.boxplot(data=comparison_df, x='Life_Stage', y=sensor, ax=ax)
            ax.set_title(f'Distribution Shift: {sensor}')

        plt.suptitle(f'Start vs. End of Life (Cycles <= {cycle_thresh} vs. RUL <= {cycle_thresh})', fontsize=20)
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()
    except Exception as e:
        print(f"  Failed plot 10: {e}")

# ---------------------------
# MAIN EXECUTION
# ---------------------------

def main():
    """Main function to run all EDA plots."""
    print("--- Starting EDA for FD003 (Extended) ---")

    if not os.path.exists(PLOT_DIR):
        try:
            os.makedirs(PLOT_DIR)
            print(f"Created directory: {PLOT_DIR}")
        except OSError as e:
            print(f"FATAL ERROR: Could not create directory: {PLOT_DIR}\n{e}")
            return

    # Load data
    train_df, test_df, rul_df, max_cycles_data = load_data()
    if train_df is None:
        print("Aborting due to data loading failure.")
        return

    # Run all 10 plot functions
    plot_1_lifespan_histogram(max_cycles_data, os.path.join(PLOT_DIR, "01_engine_lifespan_hist.png"))
    plot_2_sensor_traces(train_df, os.path.join(PLOT_DIR, "02_sensor_traces.png"))
    plot_3_correlation_heatmap(train_df, os.path.join(PLOT_DIR, "03_sensor_correlation_heatmap.png"))
    plot_4_train_test_distributions(train_df, test_df, os.path.join(PLOT_DIR, "04_train_test_sensor_dist.png"))
    plot_5_rul_target_distributions(train_df, rul_df, os.path.join(PLOT_DIR, "05_rul_target_dist.png"))
    plot_6_op_setting_validation(train_df, test_df, os.path.join(PLOT_DIR, "06_op_setting_validation.png"))

    # Plot 7 returns the top sensors for the next plots
    top_4_sensors = plot_7_rul_correlation_heatmap(train_df, os.path.join(PLOT_DIR, "07_rul_correlation_heatmap.png"))

    if top_4_sensors:
        plot_8_sensor_rul_scatter(train_df, top_4_sensors, os.path.join(PLOT_DIR, "08_sensor_rul_scatter.png"))
        plot_9_normalized_degradation(train_df, top_4_sensors, os.path.join(PLOT_DIR, "09_normalized_degradation.png"))
        plot_10_life_start_vs_end_boxplots(train_df, top_4_sensors, os.path.join(PLOT_DIR, "10_start_vs_end_boxplots.png"))
    else:
        print("Skipping plots 8, 9, 10 because RUL correlation failed.")

    print("\n--- EDA Plot Generation Complete ---")
    print(f"All plots saved to '{PLOT_DIR}'")

if __name__ == "__main__":
    main()

--- Starting EDA for FD003 (Extended) ---
Created directory: eda_plots_fd003
Loading data...
Training data shape: (24720, 28)
Test data shape: (16596, 26)
1. Plotting engine lifespan distribution...
2. Plotting sensor traces for 5 sample engines...
3. Plotting sensor correlation heatmap...
4. Plotting Train vs. Test sensor distributions...
5. Plotting RUL target distributions...
6. Plotting Operating Setting validation...
7. Plotting Sensor vs. RUL correlation heatmap...


/tmp/ipython-input-2576906455.py:242: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=rul_corr.values, y=rul_corr.index, palette='vlag')


8. Plotting scatter plots for top 4 sensors vs. RUL...
9. Plotting normalized degradation traces...
10. Plotting Start vs. End of Life distributions...

--- EDA Plot Generation Complete ---
All plots saved to 'eda_plots_fd003'


In [2]:
#!/usr/bin/env python3
"""
BLOCK 1 (V6 Refactored, V-User Re-coupled): Model Training and Artifact Generation
This block runs the full ML pipeline, trains the model, and saves all
necessary artifacts (model, scaler, predictions, history).

IT ALSO includes the 10-engine hardcoded demo from the original script,
as requested.
"""

import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from scipy.stats import norm
import joblib # For saving the scaler
import json # For saving history

# ---------------------------
# CONSTANTS / CONFIG
# ---------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

WINDOW_SIZE = 50
RUL_CAP = 125
SENSOR_COLS = [f's_{i}' for i in range(1, 22)]
OP_SETTING_COLS = [f'os_{i}' for i in range(1, 4)]

DROP_SENSORS = ['s_1', 's_5', 's_10', 's_16', 's_18', 's_19']
FEATURE_COLS = [c for c in SENSOR_COLS + OP_SETTING_COLS if c not in DROP_SENSORS]

EPS = 1e-8
L2_LAMBDA = 1e-4

# --- ECONOMIC MODEL PARAMETERS (Copied from original) ---
FAILURE_PENALTY = 5000000.0
FIXED_REVENUE = 500000.0
MAINTENANCE_COST = 100000.0
FIXED_OP_COST = 10000.0
DETERM_BUFFER = 20

# Training defaults
INITIAL_EPOCHS = 100
BATCH_SIZE = 32
MC_RUNS = 300
LEARNING_RATE = 1e-3
PATIENCE = 6

# --- Artifact Filenames ---
MODEL_FILE = 'fd003_model.keras'
SCALER_FILE = 'fd003_scaler.joblib'
HISTORY_FILE = 'fd003_train_history.json'
TEST_DATA_FILE = 'fd003_test_sequences.npz'
VAL_PREDS_FILE = 'fd003_val_preds.npz'
TEST_PREDS_FILE = 'fd003_test_preds.npz'
TEST_META_FILE = 'fd003_test_meta.csv'

# ---------------------------
# TENSORFLOW GPU SETUP
# ---------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
        print("TensorFlow initialized with GPU Strategy.")
    except RuntimeError as e:
        print(f"TensorFlow GPU setup failed: {e}")
        strategy = tf.distribute.get_strategy()
else:
    strategy = tf.distribute.get_strategy()
    print("No GPU detected. Falling back to default CPU Strategy.")


# ---------------------------
# UTILITY AND PREP FUNCTIONS
# ---------------------------
def safe_read_cmapss(path):
    return pd.read_csv(path, sep=r'\s+', header=None, engine='python')

def engineer_features(df, sensor_cols, window=10):
    available_sensors = [s for s in sensor_cols if s not in DROP_SENSORS and s in df.columns]
    new_features = []

    for eid in df['engine_id'].unique():
        engine_df = df[df['engine_id'] == eid].copy()
        for sensor in available_sensors:
            engine_df[f'{sensor}_mean_{window}c'] = engine_df[sensor].rolling(window=window, min_periods=1).mean()
            engine_df[f'{sensor}_std_{window}c'] = engine_df[sensor].rolling(window=window, min_periods=1).std().fillna(0)
            if eid == df['engine_id'].iloc[0]:
                new_features.extend([f'{sensor}_mean_{window}c', f'{sensor}_std_{window}c'])
        df.loc[engine_df.index] = engine_df
    return df, list(set(new_features))

def load_and_label(train_file, test_file, rul_file):
    col_names = ['engine_id', 'cycle'] + OP_SETTING_COLS + SENSOR_COLS
    train_raw = safe_read_cmapss(train_file)
    test_raw = safe_read_cmapss(test_file)
    rul_raw = safe_read_cmapss(rul_file)
    train_raw.columns = col_names[:train_raw.shape[1]]
    test_raw.columns = col_names[:test_raw.shape[1]]

    train_max = train_raw.groupby('engine_id')['cycle'].max().rename('max_cycle')
    train = train_raw.merge(train_max, left_on='engine_id', right_index=True)
    train['RUL'] = (train['max_cycle'] - train['cycle']).clip(upper=RUL_CAP).astype(float)
    train.drop(columns=['max_cycle'], inplace=True)

    numeric_cols = rul_raw.select_dtypes(include=[np.number]).columns
    rul_vals = rul_raw[numeric_cols[0]].values if len(numeric_cols) > 0 else rul_raw.iloc[:, 0].values
    test_engine_ids = np.sort(test_raw['engine_id'].unique())
    if len(test_engine_ids) != len(rul_vals):
        test_engine_ids = np.arange(1, len(rul_vals) + 1)
    rul_map = dict(zip(test_engine_ids, rul_vals.astype(float)))

    test_last = test_raw.groupby('engine_id')['cycle'].max().reset_index().rename(columns={'cycle':'last_cycle'})
    test_last['RUL_last'] = test_last['engine_id'].map(rul_map).astype(float)
    test = test_raw.merge(test_last[['engine_id','last_cycle','RUL_last']], on='engine_id', how='left')
    test['RUL'] = (test['RUL_last'] + (test['last_cycle'] - test['cycle'])).clip(upper=RUL_CAP).astype(float)
    test.drop(columns=['last_cycle','RUL_last'], inplace=True)

    train, added_features_train = engineer_features(train, SENSOR_COLS)
    test, added_features_test = engineer_features(test, SENSOR_COLS)

    all_features = FEATURE_COLS + list(set(added_features_train + added_features_test))
    available_features = [c for c in all_features if c in train.columns and c in test.columns]

    if len(available_features) == 0:
        raise RuntimeError("No feature columns available after filtering; check file format and DROP_SENSORS.")

    train = train[['engine_id','cycle','RUL'] + available_features].copy()
    test  = test[['engine_id','cycle','RUL'] + available_features].copy()
    return train, test, available_features

def engine_level_split(train_df, val_frac=0.2):
    engine_ids = np.sort(train_df['engine_id'].unique())
    rng = np.random.RandomState(SEED)
    shuffled = engine_ids.copy()
    rng.shuffle(shuffled)
    split = int(len(shuffled) * (1 - val_frac))
    train_ids = shuffled[:split]
    val_ids   = shuffled[split:]
    train_part = train_df[train_df['engine_id'].isin(train_ids)].copy()
    val_part   = train_df[train_df['engine_id'].isin(val_ids)].copy()
    return train_part, val_part, train_ids, val_ids

def create_sequences(df, feature_cols, window_size=WINDOW_SIZE):
    X_list, y_list, meta = [], [], []
    seq_idx = 0
    for eid in np.sort(df['engine_id'].unique()):
        sub = df[df['engine_id'] == eid].sort_values('cycle')
        n = sub.shape[0]
        if n < window_size:
            continue
        feats = sub[feature_cols].values.astype(float)
        ruls  = sub['RUL'].values.astype(float)
        for i in range(n - window_size + 1):
            X_list.append(feats[i:i+window_size])
            y_list.append(ruls[i+window_size-1])
            seq_idx += 1
        # Store the index of the *last* window for this engine
        last_window_start = seq_idx - 1
        meta.append({'engine_id': int(eid), 'seq_index_of_last_window': int(last_window_start)})
    if len(X_list) == 0:
        return np.empty((0, window_size, len(feature_cols))), np.empty((0,)), pd.DataFrame(meta)
    X = np.stack(X_list, axis=0)
    y = np.array(y_list, dtype=float)
    return X, y, pd.DataFrame(meta)

def fit_scaler(df, feature_cols):
    scaler = StandardScaler()
    scaler.fit(df[feature_cols].values)
    return scaler

def apply_scaler(df, feature_cols, scaler):
    arr = (df[feature_cols].values - scaler.mean_) / (scaler.scale_ + EPS)
    arr = np.nan_to_num(arr, posinf=0.0, neginf=0.0)
    df_scaled = df.copy()
    df_scaled[feature_cols] = arr
    return df_scaled

# ---------------------------
# MODEL DEFINITIONS
# ---------------------------
def build_model(input_shape, lstm1=200, lstm2=100, lstm3=50, dropout_rate=0.3):
    inputs = Input(shape=input_shape, name='input')
    x = LSTM(lstm1, return_sequences=True, name='lstm1', kernel_regularizer=l2(L2_LAMBDA))(inputs)
    x = Dropout(0.2)(x)
    x = LSTM(lstm2, return_sequences=True, name='lstm2', kernel_regularizer=l2(L2_LAMBDA))(x)
    x = Dropout(0.2)(x)
    x = LSTM(lstm3, return_sequences=False, name='lstm3', kernel_regularizer=l2(L2_LAMBDA))(x)
    x = Dropout(dropout_rate)(x, training=True)
    mu = Dense(1, activation='relu', name='mu')(x)
    log_sigma_sq = Dense(1, activation='linear', name='log_sigma')(x)
    out = concatenate([mu, log_sigma_sq], name='out')
    model = Model(inputs=inputs, outputs=out, name='optimized_mc_dropout_lstm')
    return model

def gaussian_nll(y_true, y_pred):
    y_flat = tf.reshape(y_true, [-1])
    mu = tf.reshape(y_pred[:,0], [-1])
    log_sigma_sq = tf.reshape(y_pred[:,1], [-1])
    sigma_sq = tf.exp(log_sigma_sq) + EPS
    nll = 0.5 * log_sigma_sq + 0.5 * tf.square(y_flat - mu) / sigma_sq
    return tf.reduce_mean(nll)

def train_with_val(X_train, y_train, X_val, y_val, epochs=INITIAL_EPOCHS, batch_size=BATCH_SIZE):
    with strategy.scope():
        model = build_model(input_shape=(X_train.shape[1], X_train.shape[2]))
        model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss=gaussian_nll)
    es = EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True, verbose=1)
    history = model.fit(X_train, y_train.reshape(-1,1),
                        validation_data=(X_val, y_val.reshape(-1,1)),
                        epochs=epochs, batch_size=batch_size, callbacks=[es], verbose=2)
    best_epochs = es.stopped_epoch if es.stopped_epoch > 0 else epochs
    return model, best_epochs, history

def retrain_on_full(trainval_X, trainval_y, epochs, batch_size=BATCH_SIZE):
    with strategy.scope():
        model = build_model(input_shape=(trainval_X.shape[1], trainval_X.shape[2]))
        model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss=gaussian_nll)
    model.fit(trainval_X, trainval_y.reshape(-1,1), epochs=epochs, batch_size=batch_size, verbose=2)
    return model

def mc_predict(model, X, n_mc=MC_RUNS):
    preds = []
    for _ in range(n_mc):
        out = model(X, training=True).numpy()
        preds.append(out)
    preds = np.stack(preds, axis=0) # Shape: (n_mc, n_samples, 2)

    mu_mc = preds[..., 0] # Shape: (n_mc, n_samples)
    log_sigma_sq_mc = preds[..., 1] # Shape: (n_mc, n_samples)

    mu_mean = np.mean(mu_mc, axis=0).flatten() # Shape: (n_samples,)

    aleatoric_var = np.mean(np.exp(log_sigma_sq_mc), axis=0).flatten() # Shape: (n_samples,)
    epistemic_var = np.var(mu_mc, axis=0).flatten() # Shape: (n_samples,)

    total_var = aleatoric_var + epistemic_var
    total_sigma = np.sqrt(total_var + EPS)

    return mu_mean, total_sigma, aleatoric_var, epistemic_var

def evaluate_prob(y_true, mu_preds, sigma_preds, confidence=0.95):
    rmse = np.sqrt(mean_squared_error(y_true, mu_preds))
    alpha = 1 - confidence
    z = norm.ppf(1 - alpha / 2.0)
    lower = mu_preds - z * sigma_preds
    upper = mu_preds + z * sigma_preds
    coverage = np.mean((y_true >= lower) & (y_true <= upper))
    mpiw = np.mean(upper - lower)
    return rmse, coverage, mpiw, lower, upper

# ---------------------------
# ASSIGNMENT FUNCTIONS (From Original Script)
# ---------------------------
def survival_prob(mu, sigma, route_length):
    sigma_safe = np.maximum(sigma, 1e-6)
    return 1.0 - norm.cdf(route_length, loc=mu, scale=sigma_safe)

def expected_profit(decision_type, p_surv=None, pred_rul=None):
    cpc = MAINTENANCE_COST / RUL_CAP

    if decision_type == 'MAINTENANCE':
        if pred_rul is None:
            return 0.0
        else:
            return -(cpc * pred_rul)

    elif decision_type == 'ASSIGNED' and p_surv is not None:
        p_fail = 1.0 - p_surv
        profit_if_success = FIXED_REVENUE - FIXED_OP_COST
        loss_if_failure = -(FIXED_OP_COST + FAILURE_PENALTY)
        e_pi_assign = (p_surv * profit_if_success) + (p_fail * loss_if_failure)
        return e_pi_assign
    return 0.0

def deterministic_assign(engines, routes):
    eng_sorted = sorted(engines, key=lambda e: e['pred_rul'], reverse=True)
    routes_sorted = sorted(routes, key=lambda r: r['length'], reverse=True)
    assigned = set()
    assignments = []
    total_expected_profit = 0.0

    for r in routes_sorted:
        chosen = None
        for e in eng_sorted:
            if e['id'] in assigned: continue
            if e['pred_rul'] >= r['length'] + DETERM_BUFFER:
                chosen = e
                break

        if chosen:
            p_surv = survival_prob(chosen['pred_rul'], chosen['pred_sigma'], r['length'])
            e_pi = expected_profit('ASSIGNED', p_surv=p_surv)

            assignments.append({
                'route':r['id'], 'engine':chosen['id'], 'status':'ASSIGNED_DETERM',
                'pred_rul':chosen['pred_rul'], 'p_survival':p_surv, 'e_profit':e_pi
            })
            total_expected_profit += e_pi
            assigned.add(chosen['id'])

    for e in eng_sorted:
        if e['id'] not in assigned:
            e_pi = expected_profit('MAINTENANCE', pred_rul=e['pred_rul'])
            assignments.append({
                'route':'N/A', 'engine':e['id'], 'status':'MAINTENANCE',
                'pred_rul':e['pred_rul'], 'p_survival':'N/A', 'e_profit':e_pi
            })
            total_expected_profit += e_pi
    return assignments, total_expected_profit

def probabilistic_assign(engines, routes):
    engine_maintenance_profit = {e['id']: expected_profit('MAINTENANCE', pred_rul=e['pred_rul']) for e in engines}
    profit_matrix = {}
    for e in engines:
        e['route_probs'] = {r['id']: survival_prob(e['pred_rul'], e['pred_sigma'], r['length']) for r in routes}
        profit_matrix[e['id']] = {}
        for r in routes:
            p_surv = e['route_probs'][r['id']]
            profit_matrix[e['id']][r['id']] = expected_profit('ASSIGNED', p_surv=p_surv)

    eng_sorted = sorted(engines, key=lambda e: e['pred_rul'], reverse=True)
    routes_sorted = sorted(routes, key=lambda r: r['length'], reverse=True)
    assigned_engines = set()
    assignments = []
    total_expected_profit = 0.0

    for r in routes_sorted:
        best_e = None
        max_profit_found = float('-inf')

        for e in eng_sorted:
            if e['id'] in assigned_engines: continue

            e_pi_assign = profit_matrix[e['id']][r['id']]
            e_pi_maint = engine_maintenance_profit[e['id']]

            if e_pi_assign > e_pi_maint and e_pi_assign > max_profit_found:
                max_profit_found = e_pi_assign
                best_e = e

        if best_e:
            p_survival = best_e['route_probs'][r['id']]
            assignments.append({'route':r['id'], 'engine':best_e['id'], 'status':'ASSIGNED_PROB',
                                'pred_rul':best_e['pred_rul'], 'p_survival':p_survival, 'e_profit':max_profit_found})
            total_expected_profit += max_profit_found
            assigned_engines.add(best_e['id'])

    for e in eng_sorted:
        if e['id'] not in assigned_engines:
            e_pi = engine_maintenance_profit[e['id']]
            assignments.append({'route':'N/A', 'engine':e['id'], 'status':'MAINTENANCE',
                                'pred_rul':e['pred_rul'], 'p_survival':'N/A', 'e_profit':e_pi})
            total_expected_profit += e_pi

    return assignments, total_expected_profit


# ---------------------------
# MAIN EXECUTION BLOCK (Block 1)
# ---------------------------
def run_block_1_training():
    TRAIN_FILE = "train_FD003.txt"
    TEST_FILE  = "test_FD003.txt"
    RUL_FILE   = "RUL_FD003.txt"

    if not (os.path.exists(TRAIN_FILE) and os.path.exists(TEST_FILE) and os.path.exists(RUL_FILE)):
        print("ERROR: One or more required files are missing.")
        return

    print("=== BLOCK 1: MODEL TRAINING AND ARTIFACT GENERATION ===")

    # 1. Load and Preprocess
    print("Loading and engineering features...")
    train_df, test_df, feature_cols = load_and_label(TRAIN_FILE, TEST_FILE, RUL_FILE)
    train_part, val_part, _, _ = engine_level_split(train_df, val_frac=0.2)

    # 2. Fit Scaler and Apply
    print("Fitting scaler on training data...")
    scaler = fit_scaler(train_part, feature_cols)
    train_scaled = apply_scaler(train_part, feature_cols, scaler)
    val_scaled   = apply_scaler(val_part,   feature_cols, scaler)
    test_scaled  = apply_scaler(test_df,    feature_cols, scaler)

    # 3. Create Sequences
    print("Creating sequences...")
    X_train, y_train, _ = create_sequences(train_scaled, feature_cols, WINDOW_SIZE)
    X_val,   y_val,   _ = create_sequences(val_scaled,   feature_cols, WINDOW_SIZE)
    X_test,  y_test,  test_meta = create_sequences(test_scaled,  feature_cols, WINDOW_SIZE)

    print(f"Sequences -> X_train: {X_train.shape}, X_val: {X_val.shape}, X_test: {X_test.shape}")

    # 4. Train with Validation
    print("Training model with validation split...")
    model_val, best_epochs, history = train_with_val(X_train, y_train, X_val, y_val,
                                                     epochs=INITIAL_EPOCHS, batch_size=BATCH_SIZE)

    # 5. Retrain on Full Data
    print(f"Retraining on full dataset for {best_epochs} epochs...")
    trainval_df = pd.concat([train_part, val_part], axis=0).reset_index(drop=True)
    trainval_scaled = apply_scaler(trainval_df, feature_cols, scaler)
    X_trainval, y_trainval, _ = create_sequences(trainval_scaled, feature_cols, WINDOW_SIZE)

    if X_trainval.shape[0] == 0:
        print("Warning: Full training set is empty. Using validation model.")
        model_final = model_val
    else:
        model_final = retrain_on_full(X_trainval, y_trainval, epochs=best_epochs, batch_size=BATCH_SIZE)

    # 6. Generate Predictions
    print(f"Generating MC-Dropout predictions (N={MC_RUNS})...")

    # Validation set predictions (for plotting overfitting)
    mu_val, sig_val, al_var_val, ep_var_val = mc_predict(model_final, X_val, n_mc=MC_RUNS)

    # Test set predictions (for simulation)
    mu_test, sig_test, al_var_test, ep_var_test = mc_predict(model_final, X_test, n_mc=MC_RUNS)

    # 7. Evaluate and Print Metrics
    rmse, picp, mpiw, _, _ = evaluate_prob(y_test, mu_test, sig_test, confidence=0.95)
    print("\n--- TEST EVALUATION (Phase 1 Success Metric) ---")
    print(f"RMSE (Accuracy of Mean): {rmse:.3f} cycles")
    print(f"PICP (95% Coverage): {picp:.3f} (Target: ~0.95)")
    print(f"MPIW (Interval Width): {mpiw:.3f} cycles")

    # 8. --- RE-INJECTED 10-ENGINE ASSIGNMENT (From Original) ---
    print("\n--- DETAILED 10-ENGINE ASSIGNMENT COMPARISON ---")

    # Sample 10 engines
    sample_meta = test_meta.sample(n=min(10, len(test_meta)), random_state=SEED).reset_index(drop=True)
    engines = []
    for _, row in sample_meta.iterrows():
        seq_idx = int(row['seq_index_of_last_window'])
        engines.append({'id': f"E{int(row['engine_id']):03d}",
                        'pred_rul': float(mu_test[seq_idx]),     # Use my mu_test
                        'pred_sigma': float(sig_test[seq_idx])}) # Use my sig_test

    # Hardcoded routes from original script
    route_lengths = [30, 45, 60, 75, 90, 100, 110, 120, 35, 85]
    routes = [{'id': f'R{i+1:02d}', 'length': route_lengths[i]} for i in range(min(len(route_lengths), len(engines)))]

    # Run both assignment strategies
    det_assign, det_profit = deterministic_assign(engines, routes)
    prob_assign, prob_profit = probabilistic_assign(engines, routes)

    print(f"Revenue: ${FIXED_REVENUE:,.0f} | Failure Penalty: ${FAILURE_PENALTY:,.0f} | Maint Cost (Wasted Life CPC): ${MAINTENANCE_COST:,.0f}")

    print("\nDeterministic Assignment Results (RUL+Buffer Rule, but E-Profit Costed):")
    print(pd.DataFrame(det_assign).to_markdown(index=False))
    print(f"Total Deterministic E-Profit: ${det_profit:,.2f}")

    print("\nProbabilistic Assignment Results (Maximizing E-Profit):")
    print(pd.DataFrame(prob_assign).to_markdown(index=False))
    print(f"Total Probabilistic E-Profit: ${prob_profit:,.2f}")
    # --- END OF RE-INJECTED BLOCK ---


    # 9. Save All Artifacts
    print("\nSaving artifacts...")
    model_final.save(MODEL_FILE)
    print(f"Model saved to: {MODEL_FILE}")

    joblib.dump(scaler, SCALER_FILE)
    print(f"Scaler saved to: {SCALER_FILE}")

    with open(HISTORY_FILE, 'w') as f:
        json.dump(history.history, f)
    print(f"Training history saved to: {HISTORY_FILE}")

    # Save test data sequences
    np.savez_compressed(TEST_DATA_FILE, X_test=X_test, y_test=y_test)
    print(f"Test sequences saved to: {TEST_DATA_FILE}")

    # Save test metadata
    test_meta.to_csv(TEST_META_FILE, index=False)
    print(f"Test metadata saved to: {TEST_META_FILE}")

    # Save validation predictions
    np.savez_compressed(VAL_PREDS_FILE,
                        mu=mu_val, sigma=sig_val,
                        aleatoric_var=al_var_val, epistemic_var=ep_var_val,
                        y_val=y_val)
    print(f"Validation predictions saved to: {VAL_PREDS_FILE}")

    # Save test predictions
    np.savez_compressed(TEST_PREDS_FILE,
                        mu=mu_test, sigma=sig_test,
                        aleatoric_var=al_var_test, epistemic_var=ep_var_test)
    print(f"Test predictions saved to: {TEST_PREDS_FILE}")

    print("\n=== BLOCK 1 COMPLETE ===")

if __name__ == "__main__":
    run_block_1_training()

TensorFlow initialized with GPU Strategy.
=== BLOCK 1: MODEL TRAINING AND ARTIFACT GENERATION ===
Loading and engineering features...
Fitting scaler on training data...
Creating sequences...
Sequences -> X_train: (16257, 50, 18), X_val: (3563, 50, 18), X_test: (11717, 50, 18)
Training model with validation split...
Epoch 1/100
509/509 - 11s - 22ms/step - loss: 44.0204 - val_loss: 5.0086
Epoch 2/100
509/509 - 7s - 14ms/step - loss: 5.3288 - val_loss: 4.9788
Epoch 3/100
509/509 - 6s - 12ms/step - loss: 5.2086 - val_loss: 4.9254
Epoch 4/100
509/509 - 8s - 15ms/step - loss: 5.1410 - val_loss: 4.8455
Epoch 5/100
509/509 - 6s - 12ms/step - loss: 5.0502 - val_loss: 4.7431
Epoch 6/100
509/509 - 7s - 14ms/step - loss: 4.9238 - val_loss: 4.6069
Epoch 7/100
509/509 - 8s - 15ms/step - loss: 4.8050 - val_loss: 4.4430
Epoch 8/100
509/509 - 7s - 14ms/step - loss: 4.6660 - val_loss: 4.4316
Epoch 9/100
509/509 - 7s - 14ms/step - loss: 4.5509 - val_loss: 4.5112
Epoch 10/100
509/509 - 6s - 12ms/step - lo

In [3]:
#!/usr/bin/env python3
"""
BLOCK 2 (V-User): STATISTICAL VALIDATION (100 Iterations)

This block loads the predictions and metadata from Block 1,
runs the 100-iteration statistical simulation, prints the
full results to the console (as requested), and saves
the results to a CSV.
"""

import os
import random
import numpy as np
import pandas as pd
from scipy.stats import norm

# ---------------------------
# CONFIGURATION
# ---------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# --- Economic Constants (Must match Block 1) ---
FAILURE_PENALTY = 5000000.0
FIXED_REVENUE = 500000.0
MAINTENANCE_COST = 100000.0
FIXED_OP_COST = 10000.0
DETERM_BUFFER = 20
RUL_CAP = 125 # Used for maintenance cost calculation

# --- Simulation Parameters ---
# Using 100 iterations / 10 engines as per your *original code*
# (Your log file shows 200/50, which is a mismatch)
NUM_ITERATIONS = 200
NUM_ENGINES_PER_ITER = 30
ROUTE_RUL_MIN = 25
ROUTE_RUL_MAX = 125

# --- Input Artifacts (from Block 1) ---
TEST_PREDS_FILE = 'fd003_test_preds.npz'
TEST_META_FILE = 'fd003_test_meta.csv'

# --- Output Artifact ---
SIMULATION_CSV_FILE = 'fd003_simulation_results.csv'

# ---------------------------
# ASSIGNMENT FUNCTIONS
# (Copied from Block 1 to make this script standalone)
# ---------------------------

def survival_prob(mu, sigma, route_length):
    sigma_safe = np.maximum(sigma, 1e-6)
    return 1.0 - norm.cdf(route_length, loc=mu, scale=sigma_safe)

def expected_profit(decision_type, p_surv=None, pred_rul=None):
    cpc = MAINTENANCE_COST / RUL_CAP

    if decision_type == 'MAINTENANCE':
        if pred_rul is None:
            return 0.0
        else:
            return -(cpc * pred_rul)

    elif decision_type == 'ASSIGNED' and p_surv is not None:
        p_fail = 1.0 - p_surv
        profit_if_success = FIXED_REVENUE - FIXED_OP_COST
        loss_if_failure = -(FIXED_OP_COST + FAILURE_PENALTY)
        e_pi_assign = (p_surv * profit_if_success) + (p_fail * loss_if_failure)
        return e_pi_assign
    return 0.0

def deterministic_assign(engines, routes):
    eng_sorted = sorted(engines, key=lambda e: e['pred_rul'], reverse=True)
    routes_sorted = sorted(routes, key=lambda r: r['length'], reverse=True)
    assigned = set()
    assignments = []
    total_expected_profit = 0.0

    for r in routes_sorted:
        chosen = None
        for e in eng_sorted:
            if e['id'] in assigned: continue
            if e['pred_rul'] >= r['length'] + DETERM_BUFFER:
                chosen = e
                break

        if chosen:
            p_surv = survival_prob(chosen['pred_rul'], chosen['pred_sigma'], r['length'])
            e_pi = expected_profit('ASSIGNED', p_surv=p_surv)
            assignments.append({
                'route':r['id'], 'engine':chosen['id'], 'status':'ASSIGNED_DETERM',
                'pred_rul':chosen['pred_rul'], 'p_survival':p_surv, 'e_profit':e_pi
            })
            total_expected_profit += e_pi
            assigned.add(chosen['id'])

    for e in eng_sorted:
        if e['id'] not in assigned:
            e_pi = expected_profit('MAINTENANCE', pred_rul=e['pred_rul'])
            assignments.append({
                'route':'N/A', 'engine':e['id'], 'status':'MAINTENANCE',
                'pred_rul':e['pred_rul'], 'p_survival':'N/A', 'e_profit':e_pi
            })
            total_expected_profit += e_pi
    return assignments, total_expected_profit

def probabilistic_assign(engines, routes):
    engine_maintenance_profit = {e['id']: expected_profit('MAINTENANCE', pred_rul=e['pred_rul']) for e in engines}
    profit_matrix = {}
    for e in engines:
        e['route_probs'] = {r['id']: survival_prob(e['pred_rul'], e['pred_sigma'], r['length']) for r in routes}
        profit_matrix[e['id']] = {}
        for r in routes:
            p_surv = e['route_probs'][r['id']]
            profit_matrix[e['id']][r['id']] = expected_profit('ASSIGNED', p_surv=p_surv)

    eng_sorted = sorted(engines, key=lambda e: e['pred_rul'], reverse=True)
    routes_sorted = sorted(routes, key=lambda r: r['length'], reverse=True)
    assigned_engines = set()
    assignments = []
    total_expected_profit = 0.0

    for r in routes_sorted:
        best_e = None
        max_profit_found = float('-inf')

        for e in eng_sorted:
            if e['id'] in assigned_engines: continue
            e_pi_assign = profit_matrix[e['id']][r['id']]
            e_pi_maint = engine_maintenance_profit[e['id']]

            if e_pi_assign > e_pi_maint and e_pi_assign > max_profit_found:
                max_profit_found = e_pi_assign
                best_e = e

        if best_e:
            p_survival = best_e['route_probs'][r['id']]
            assignments.append({'route':r['id'], 'engine':best_e['id'], 'status':'ASSIGNED_PROB',
                                'pred_rul':best_e['pred_rul'], 'p_survival':p_survival, 'e_profit':max_profit_found})
            total_expected_profit += max_profit_found
            assigned_engines.add(best_e['id'])

    for e in eng_sorted:
        if e['id'] not in assigned_engines:
            e_pi = engine_maintenance_profit[e['id']]
            assignments.append({'route':'N/A', 'engine':e['id'], 'status':'MAINTENANCE',
                                'pred_rul':e['pred_rul'], 'p_survival':'N/A', 'e_profit':e_pi})
            total_expected_profit += e_pi
    return assignments, total_expected_profit

# ---------------------------
# STATISTICAL SIMULATION DRIVER
# ---------------------------
def run_simulation(mu_preds, sigma_preds, test_meta):
    """Runs N randomized iterations of the assignment simulation."""

    # Map last-window indices to engine data (RUL, Sigma)
    last_window_data = []
    test_meta_index = test_meta['seq_index_of_last_window'].tolist()
    test_engine_ids = test_meta['engine_id'].tolist()

    for i, seq_idx in enumerate(test_meta_index):
        if seq_idx < len(mu_preds):
            last_window_data.append({
                'id': f"E{test_engine_ids[i]:03d}",
                'pred_rul': float(mu_preds[seq_idx]),
                'pred_sigma': float(sigma_preds[seq_idx])
            })
        else:
            print(f"Warning: seq_idx {seq_idx} out of bounds. Skipping.")

    if len(last_window_data) < NUM_ENGINES_PER_ITER:
        raise RuntimeError(f"Insufficient test engines ({len(last_window_data)}) for required simulation size ({NUM_ENGINES_PER_ITER}).")

    simulation_results = []

    # Use a different seed block for Block 2
    rng = np.random.RandomState(SEED + 100)

    for i in range(NUM_ITERATIONS):
        # 1. Randomly select engines
        sample_engines_data = [
            last_window_data[i] for i in
            rng.choice(len(last_window_data), NUM_ENGINES_PER_ITER, replace=False)
        ]

        # 2. Randomly decide route lengths
        route_lengths = rng.randint(ROUTE_RUL_MIN, ROUTE_RUL_MAX + 1, size=NUM_ENGINES_PER_ITER)
        routes = [{'id': f'R{j+1:02d}', 'length': int(route_lengths[j])} for j in range(NUM_ENGINES_PER_ITER)]

        # 3. Run assignments
        det_assign, det_profit = deterministic_assign(sample_engines_data, routes)
        prob_assign, prob_profit = probabilistic_assign(sample_engines_data, routes)

        # Calculate percentage difference
        profit_diff = prob_profit - det_profit
        if det_profit != 0:
            perc_gain = (profit_diff / abs(det_profit)) * 100
        else:
            perc_gain = np.nan

        # Store results
        simulation_results.append({
            'iteration': i + 1,
            'route_lengths': [r['length'] for r in routes],
            'engines_used': [e['id'] for e in sample_engines_data],
            'det_profit': det_profit,
            'prob_profit': prob_profit,
            'profit_increase_abs': profit_diff,
            'profit_increase_perc': perc_gain
        })

    df_results = pd.DataFrame(simulation_results)

    # Calculate final averages
    avg_det_profit = df_results['det_profit'].mean()
    avg_prob_profit = df_results['prob_profit'].mean()
    avg_perc_gain = ((avg_prob_profit - avg_det_profit) / abs(avg_det_profit)) * 100 if avg_det_profit != 0 else np.nan # FIX: Use totals, not mean of percentages

    return df_results, avg_det_profit, avg_prob_profit, avg_perc_gain

# ---------------------------
# MAIN EXECUTION
# ---------------------------
def main():
    """Loads artifacts, runs simulation, and saves results."""

    print(f"=== BLOCK 2: STATISTICAL VALIDATION - {NUM_ITERATIONS} RUNS ===")

    # Check for input files
    if not os.path.exists(TEST_PREDS_FILE) or not os.path.exists(TEST_META_FILE):
        print(f"ERROR: Cannot find input files: {TEST_PREDS_FILE} or {TEST_META_FILE}")
        print("Please run Block 1 first.")
        return

    # Load data from Block 1
    try:
        preds = np.load(TEST_PREDS_FILE)
        MU_PREDS = preds['mu']
        SIGMA_PREDS = preds['sigma']
        TEST_META = pd.read_csv(TEST_META_FILE)
    except Exception as e:
        print(f"Error loading artifacts: {e}")
        return

    print(f"Running {NUM_ITERATIONS} iterations with random routes and {NUM_ENGINES_PER_ITER} random engines.")

    # Run simulation
    df_results, avg_det_profit, avg_prob_profit, avg_perc_gain = run_simulation(MU_PREDS, SIGMA_PREDS, TEST_META)

    # --- START: RE-ADDED VERBOSE PRINTING ---
    print("\nIndividual Simulation Results (ALL Iterations):")
    # Define columns to print, matching your old log
    cols_to_print = ['iteration', 'route_lengths', 'engines_used',
                     'det_profit', 'prob_profit', 'profit_increase_perc']

    # Set pandas options to print the full table (this can be very long)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)

    print(df_results[cols_to_print].to_markdown(index=False, floatfmt=".2f", numalign='right'))

    # Reset pandas options
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    # --- END: RE-ADDED VERBOSE PRINTING ---

    # Save results to CSV (the correct way)
    try:
        df_results.to_csv(SIMULATION_CSV_FILE, index=False) # FIX: Removed bad 'floatfmt' argument
        print(f"\nFull simulation results saved to: {SIMULATION_CSV_FILE}")
    except Exception as e:
        print(f"\nWarning: Could not save CSV file. {e}")


    # Print final summary
    print("\n--- FINAL AVERAGE PROFIT COMPARISON ---")
    print(f"Average Deterministic E-Profit: ${avg_det_profit:,.2f}")
    print(f"Average Probabilistic E-Profit: ${avg_prob_profit:,.2f}")

    if avg_perc_gain > 0:
        print(f"\nFINAL RESULT: Probabilistic approach demonstrated a robust average profit increase of {avg_perc_gain:.2f}% across all scenarios.")
    elif avg_perc_gain < 0:
        print(f"\nFINAL RESULT: Deterministic approach unexpectedly performed better on average, resulting in an average loss of {-avg_perc_gain:.2f}%.")
    else:
        print("\nFINAL RESULT: Average Expected Profits were neutral.")

    print("\n=== STATISTICAL TEST COMPLETE ===")

if __name__ == "__main__":
    main()

=== BLOCK 2: STATISTICAL VALIDATION - 200 RUNS ===
Running 200 iterations with random routes and 30 random engines.

Individual Simulation Results (ALL Iterations):
|   iteration | route_lengths                                                                                                                           | engines_used                                                                                                                                                                                                                                     |   det_profit |   prob_profit |   profit_increase_perc |
|------------:|:----------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
#!/usr/bin/env python3
"""
BLOCK 3 (New): Plotting and Analysis
This block loads all saved artifacts from Blocks 1 and 2
to generate a comprehensive set of visualizations for
model validation and simulation analysis.
"""

import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- Artifact Filenames (Inputs) ---
HISTORY_FILE = 'fd003_train_history.json'
TEST_DATA_FILE = 'fd003_test_sequences.npz'
VAL_PREDS_FILE = 'fd003_val_preds.npz'
TEST_PREDS_FILE = 'fd003_test_preds.npz'
SIMULATION_FILE = 'fd003_simulation_results.csv'

# --- Plot Output Directory ---
PLOT_DIR = 'plots_fd003'

def load_artifacts():
    """Loads all necessary files for plotting."""
    print("Loading artifacts...")

    if not os.path.exists(HISTORY_FILE):
        print(f"Warning: Missing {HISTORY_FILE}")
        history = None
    else:
        with open(HISTORY_FILE, 'r') as f:
            history = json.load(f)

    if not os.path.exists(TEST_DATA_FILE):
        print(f"Error: Missing {TEST_DATA_FILE}. Cannot proceed.")
        return None
    else:
        test_data = np.load(TEST_DATA_FILE)
        y_test = test_data['y_test']

    if not os.path.exists(VAL_PREDS_FILE):
        print(f"Warning: Missing {VAL_PREDS_FILE}")
        val_preds = None
    else:
        val_preds = np.load(VAL_PREDS_FILE)

    if not os.path.exists(TEST_PREDS_FILE):
        print(f"Error: Missing {TEST_PREDS_FILE}. Cannot proceed.")
        return None
    else:
        test_preds = np.load(TEST_PREDS_FILE)

    if not os.path.exists(SIMULATION_FILE):
        print(f"Warning: Missing {SIMULATION_FILE}")
        df_results = None
    else:
        df_results = pd.read_csv(SIMULATION_FILE)

    print("Artifact loading complete.")
    return history, y_test, val_preds, test_preds, df_results

def plot_training_history(history, save_path):
    """Plots training and validation loss."""
    if history is None:
        return

    print("Plotting training history...")
    plt.figure(figsize=(10, 6))
    plt.plot(history['loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Model Training History (Gaussian NLL)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def plot_rul_predictions(y_true, y_pred, save_path):
    """Plots predicted RUL vs. True RUL."""
    print("Plotting RUL predictions...")
    plt.figure(figsize=(10, 10))
    # Create a hexbin plot for density
    hb = plt.hexbin(y_true, y_pred, gridsize=50, cmap='inferno', mincnt=1)
    plt.colorbar(hb, label='Point Density')

    # Add y=x line
    lims = [min(plt.xlim()[0], plt.ylim()[0]), max(plt.xlim()[1], plt.ylim()[1])]
    plt.plot(lims, lims, 'r--', label='Perfect Prediction (y=x)')

    plt.xlabel('True RUL (cycles)')
    plt.ylabel('Predicted RUL (cycles)')
    plt.title('Predicted vs. True RUL (Test Set)')
    plt.legend()
    plt.xlim(lims)
    plt.ylim(lims)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def plot_error_distribution(y_true, y_pred, save_path):
    """Plots the histogram of prediction errors."""
    print("Plotting error distribution...")
    errors = y_pred - y_true
    rmse = np.sqrt(np.mean(errors**2))

    plt.figure(figsize=(10, 6))
    sns.histplot(errors, kde=True, bins=50)
    plt.title(f'Prediction Error Distribution (Test Set)\nRMSE: {rmse:.3f} cycles')
    plt.xlabel('Error (Predicted - True)')
    plt.ylabel('Frequency')
    plt.axvline(0, color='r', linestyle='--')
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def plot_uncertainty_vs_error(y_true, y_pred, sigma_pred, save_path):
    """Plots absolute error vs. predicted uncertainty."""
    print("Plotting uncertainty vs. error...")
    abs_errors = np.abs(y_pred - y_true)

    plt.figure(figsize=(10, 10))
    hb = plt.hexbin(sigma_pred, abs_errors, gridsize=50, cmap='inferno', mincnt=1)
    plt.colorbar(hb, label='Point Density')

    # Add y=x line
    lims = [0, max(plt.xlim()[1], plt.ylim()[1])]
    plt.plot(lims, lims, 'r--', label='Ideal (Error = Sigma)')

    plt.xlabel('Predicted Sigma (Uncertainty)')
    plt.ylabel('Absolute Error')
    plt.title('Uncertainty vs. Absolute Error (Test Set)')
    plt.legend()
    plt.xlim([0, lims[1]])
    plt.ylim([0, lims[1]])
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def plot_uncertainty_components(aleatoric_var, epistemic_var, save_path):
    """Plots histograms of aleatoric vs. epistemic uncertainty."""
    print("Plotting uncertainty components...")
    aleatoric_sigma = np.sqrt(aleatoric_var)
    epistemic_sigma = np.sqrt(epistemic_var)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    sns.histplot(aleatoric_sigma, kde=True, ax=ax1, color='blue', bins=50)
    ax1.set_title('Aleatoric Uncertainty (Data Noise)')
    ax1.set_xlabel('Sigma (Aleatoric)')

    sns.histplot(epistemic_sigma, kde=True, ax=ax2, color='green', bins=50)
    ax2.set_title('Epistemic Uncertainty (Model Ignorance)')
    ax2.set_xlabel('Sigma (Epistemic)')

    fig.suptitle('Distribution of Uncertainty Components (Test Set)')
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def plot_simulation_profits(df_results, save_path):
    """Plots the distribution of profits from the simulation."""
    if df_results is None:
        return

    print("Plotting simulation profit distributions...")
    plt.figure(figsize=(12, 7))
    sns.kdeplot(df_results['det_profit'], label='Deterministic Profit', fill=True, color='red')
    sns.kdeplot(df_results['prob_profit'], label='Probabilistic Profit', fill=True, color='blue')

    avg_det = df_results['det_profit'].mean()
    avg_prob = df_results['prob_profit'].mean()

    plt.axvline(avg_det, color='red', linestyle='--', label=f'Avg. Det: ${avg_det:,.0f}')
    plt.axvline(avg_prob, color='blue', linestyle='--', label=f'Avg. Prob: ${avg_prob:,.0f}')

    plt.title(f'Profit Distribution ({len(df_results)} Simulations)')
    plt.xlabel('Total Expected Profit ($)')
    plt.ylabel('Density')
    plt.legend()
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def plot_profit_gain_vs_routes(df_results, save_path):
    """Plots profit gain vs. the average route length in the scenario."""
    if df_results is None:
        return

    print("Plotting profit gain vs. route length...")
    # Safely parse the 'route_lengths' column (it's a string representation of a list)
    try:
        df_results['avg_route_length'] = df_results['route_lengths'].apply(
            lambda x: np.mean(json.loads(x.replace("'", '"')))
        )
    except Exception as e:
        print(f"Could not parse 'route_lengths': {e}")
        return

    plt.figure(figsize=(10, 6))
    df_results['profit_diff'] = df_results['prob_profit'] - df_results['det_profit']
    sns.regplot(data=df_results, x='avg_route_length', y='profit_diff',
                scatter_kws={'alpha':0.5}, line_kws={'color':'red'})

    plt.title('Profit Gain (Probabilistic - Deterministic) vs. Scenario Difficulty')
    plt.xlabel('Average Route Length in Scenario (cycles)')
    plt.ylabel('Profit Difference ($)')
    plt.axhline(0, color='k', linestyle='--')
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()


# ---------------------------
# MAIN PLOTTING EXECUTION
# ---------------------------
def run_block_3_plotting():
    print("\n=== BLOCK 3: PLOTTING AND ANALYSIS ===")

    # Create plot directory if it doesn't exist
    if not os.path.exists(PLOT_DIR):
        os.makedirs(PLOT_DIR)
        print(f"Created directory: {PLOT_DIR}")

    # Load all data
    artifacts = load_artifacts()
    if artifacts is None or artifacts[1] is None or artifacts[3] is None:
        print("Aborting plotting due to missing critical files.")
        return

    history, y_test, val_preds, test_preds, df_results = artifacts

    # --- Model Performance Plots ---
    plot_training_history(history, os.path.join(PLOT_DIR, '01_training_history.png'))

    plot_rul_predictions(y_test, test_preds['mu'], os.path.join(PLOT_DIR, '02_rul_predictions.png'))

    plot_error_distribution(y_test, test_preds['mu'], os.path.join(PLOT_DIR, '03_error_distribution.png'))

    plot_uncertainty_vs_error(y_test, test_preds['mu'], test_preds['sigma'],
                              os.path.join(PLOT_DIR, '04_uncertainty_vs_error.png'))

    plot_uncertainty_components(test_preds['aleatoric_var'], test_preds['epistemic_var'],
                                os.path.join(PLOT_DIR, '05_uncertainty_components.png'))

    # --- Simulation Analysis Plots ---
    plot_simulation_profits(df_results, os.path.join(PLOT_DIR, '06_simulation_profits.png'))

    plot_profit_gain_vs_routes(df_results, os.path.join(PLOT_DIR, '07_profit_gain_vs_routes.png'))

    print(f"\nAll plots saved to '{PLOT_DIR}' directory.")
    print("=== PLOTTING COMPLETE ===")

if __name__ == "__main__":
    run_block_3_plotting()


=== BLOCK 3: PLOTTING AND ANALYSIS ===
Loading artifacts...
Artifact loading complete.
Plotting training history...
Plotting RUL predictions...
Plotting error distribution...
Plotting uncertainty vs. error...
Plotting uncertainty components...
Plotting simulation profit distributions...
Plotting profit gain vs. route length...

All plots saved to 'plots_fd003' directory.
=== PLOTTING COMPLETE ===
